## LSTM state encoder
P.S. This snippet uses library varsion of the learning function, you can see the visualization in the tensorboard

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import pickle
import gc
import json
import h5py

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline


# == recnn ==
import sys
sys.path.append("../../")
import recnn

cuda = torch.device('cuda')

# ---
frame_size = 10
batch_size = 25
n_epochs   = 100
plot_every = 30
step       = 0
# --- 

tqdm.pandas()

In [3]:
state_encoder = nn.LSTM(129, 256, batch_first=True).to(cuda)

# embeddgings: https://drive.google.com/open?id=1EQ_zXBR3DKpmJR3jBgLvt-xoOvArGMsL
env = recnn.data.env.SeqEnv('../../data/embeddings/ml20_pca128.pkl',
                         '../../data/ml-20m/ratings.csv', state_encoder, device=cuda)

In [4]:
def run_tests():
    batch = next(env.test_batch())
    loss = ddpg_update(batch, params, nets, optimizer,
                       cuda, debug, writer, step=step, learn=False)
    return losses

In [5]:
# === ddpg settings ===

params = {
    'gamma'      : 0.99,
    'min_value'  : -10,
    'max_value'  : 10,
    'policy_step': 10,
    'soft_tau'   : 0.001,
    
    'policy_lr'  : 1e-5,
    'value_lr'   : 1e-5,
    'actor_weight_init': 54e-2,
    'critic_weight_init': 6e-1,
}

# === end ===

In [6]:

value_net  = recnn.nn.Critic(256, 128, 256, params['critic_weight_init']).to(cuda)
policy_net = recnn.nn.Actor(256, 128, 256, params['actor_weight_init']).to(cuda)

target_value_net = recnn.nn.Critic(256, 128, 256).to(cuda)
target_policy_net = recnn.nn.Actor(256, 128, 256).to(cuda)

target_policy_net.eval()
target_value_net.eval()


recnn.utils.soft_update(value_net, target_value_net, soft_tau=1.0)
recnn.utils.soft_update(policy_net, target_policy_net, soft_tau=1.0)

pm = list(policy_net.parameters()) + list(state_encoder.parameters())
value_optimizer = recnn.optim.RAdam(value_net.parameters(),
                              lr=params['value_lr'], weight_decay=1e-2)
policy_optimizer = recnn.optim.RAdam(pm, lr=params['policy_lr'] , weight_decay=1e-2)

nets = {
    'value_net': value_net,
    'target_value_net': target_value_net,
    'policy_net': policy_net,
    'target_policy_net': target_policy_net,
}

optimizer = {
    'policy_optimizer': policy_optimizer,
    'value_optimizer':  value_optimizer
}

debug = {}
writer = SummaryWriter(log_dir='../../runs')

In [7]:
step = 0
for batch in tqdm(env.train_batch()):
        loss = recnn.nn.update.ddpg_update(batch, params, nets, optimizer,
                       cuda, debug, writer, step=step)
        #if step % 10 == 0:
        #    run_tests()

KeyboardInterrupt: 